In [ ]:
import numpy as np
import cv2 as cv
import math
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
def show_image(img, figsize=(10,10), gray=True):
    plt.figure(figsize=figsize)
    plt.imshow(img, 'gray' if gray else None)
    plt.axis('off')


In [ ]:
def calculate_mean(img_dict):
    list_of_means = []
    for key, value in img_dict.items():
        img = cv.imread(value, 0)
        list_of_means.append(np.mean(img))
    return list_of_means    

In [ ]:
#function to check cancer on the basis of mean value
def check_mean(mean_im, mean_com, mean_atype, mean_melanoma):
    abs_com = abs(mean_im - mean_com)
    abs_atype = abs(mean_im - mean_atype)
    abs_melanoma = abs(mean_im - mean_melanoma)
    if abs_com < abs_atype and abs_com < abs_melanoma:
        return 1
    elif abs_atype < abs_com and abs_atype < abs_melanoma:
        return 2
    else:
        return 3
    


In [ ]:
#function to find symmetry
def symmetry(img_dict):
    list_of_means = []
    for key, value in img_dict.items():
        # Load the image
        img = cv.imread(value, 0)      

        # Find the coordinates of the non-zero pixels
        coords = cv.findNonZero(img)
        # Get the bounding box of the non-zero pixels
        x, y, w, h = cv.boundingRect(coords)

        # Crop the image
        crop_img = img[y:y+h, x:x+w]
        #cut crop_img in half horizontally
        croph_img1 = crop_img[:, :int(crop_img.shape[1]/2)]
        croph_img2 = crop_img[:, int(((crop_img.shape[1])+1)/2):]
        #flip crop_img2
        croph_img2 = cv.flip(croph_img2, 1)
       
        #calculate mean error
        horizontal_sym = np.count_nonzero(abs(croph_img1-croph_img2))
        # error1 = np.mean(croph_img1-croph_img2)

        #cut crop_img in half vertically
        cropv_img1 = crop_img[:int(crop_img.shape[0]/2), :]
        cropv_img2 = crop_img[int(((crop_img.shape[0])+1)/2):, :]
        cropv_img2 = cv.flip(cropv_img2, 0)
        vertical_sym = np.count_nonzero(abs(cropv_img1-cropv_img2))
        # error2 = np.mean(cropv_img1-cropv_img2)

        error = np.mean([horizontal_sym, vertical_sym])
        
        list_of_means.append(error)


    return list_of_means    
    

In [ ]:
def border_irregularity(img_dict):
    list_of_means = []
    for key, value in img_dict.items():
        img = cv.imread(value, 0)
        lesion_points = np.where(img>0)
        x = math.ceil(np.mean(lesion_points[0]))
        y = math.ceil(np.mean(lesion_points[1]))
        edges = cv.Canny(img, 100, 200)
        locations = np.where(edges>0)
        max_dist = []
        for i in range(0, len(locations)):
            max_dist.append(math.sqrt((locations[0][i]-x)**2 + (locations[1][i]-y)**2))

        lesion_area = np.count_nonzero(edges)
        circle_area = math.pi * max(max_dist)**2
        list_of_means.append(1-(lesion_area/circle_area))
        
    return list_of_means

            

        

In [ ]:
#paths to images
common_nevus = {}
a_type = {}
melanoma = {}

common_nevus_color = {}
a_type_color = {}
melanoma_color = {}

df = pd.read_excel('dataset/PH2_dataset.xlsx' , usecols=[0, 2, 3, 4], skiprows=12)

for row in df.iterrows():
    # print(row[1][0])
    if row[1][1] == 'X':
        common_nevus[row[1][0]] = 'Dataset_images/' + row[1][0] + '/' + row[1][0] + '_lesion/' + row[1][0] + '_lesion' + '.bmp'
        common_nevus_color[row[1][0]] = 'Dataset_images/' + row[1][0] + '/' + row[1][0] + '_Dermoscopic_Image/' + row[1][0] + '.bmp'
    elif row[1][2] == 'X':
        a_type[row[1][0]] = 'Dataset_images/' + row[1][0] + '/' + row[1][0] + '_lesion/' + row[1][0] + '_lesion' + '.bmp'
        a_type_color[row[1][0]] = 'Dataset_images/' + row[1][0] + '/' + row[1][0] + '_Dermoscopic_Image/' + row[1][0] + '.bmp'
    elif row[1][3] == 'X':
        melanoma[row[1][0]] = 'Dataset_images/' + row[1][0] + '/' + row[1][0] + '_lesion/' + row[1][0] + '_lesion' + '.bmp'
        melanoma_color[row[1][0]] = 'Dataset_images/' + row[1][0] + '/' + row[1][0] + '_Dermoscopic_Image/' + row[1][0] + '.bmp'


In [ ]:
def accuracy(lesion_type, means_list):
    common_count = 0
    a_type_count = 0
    melanoma_count = 0
    for i in lesion_type:
        check_val = check_mean(i, means_list[0], means_list[1], means_list[2])
        if check_val == 1:
            common_count += 1
        elif check_val == 2:
            a_type_count += 1
        else:
            melanoma_count += 1
    return [common_count, a_type_count, melanoma_count]        

In [ ]:
# Diameter of lesion
mean_common = calculate_mean(common_nevus)
mean_a_type = calculate_mean(a_type)
mean_melanoma = calculate_mean(melanoma)

#box plot of above means
data = [mean_common, mean_a_type, mean_melanoma]
fig = plt.figure(figsize =(10, 7))
ax = fig.add_axes([0, 0, 1, 1])
ax.set_title('Mean Values for Diameter of Common, A-Type and Melanoma')
ax.set_xticklabels(['Common', 'A-Type', 'Melanoma'])
bp = ax.boxplot(data)
plt.show()
#calculating the mean of the means
mean_com = np.mean(mean_common)
mean_atype = np.mean(mean_a_type)
mean_melanom = np.mean(mean_melanoma)


dia_accuracy_common = accuracy(mean_common, [mean_com, mean_atype, mean_melanom])
dia_accuracy_a_type = accuracy(mean_a_type, [mean_com, mean_atype, mean_melanom])
dia_accuracy_melanoma = accuracy(mean_melanoma, [mean_com, mean_atype, mean_melanom])

print('Accuracy for Common Nevus: ', (dia_accuracy_common[0]/len(mean_common)) * 100, '%')
print('Accuracy for A-Type Nevus: ', (dia_accuracy_a_type[1]/len(mean_a_type))*100, '%')
print('Accuracy for Melanoma: ', (dia_accuracy_melanoma[2]/len(mean_melanoma))*100, '%')


In [ ]:
#symmetry
sym_common = symmetry(common_nevus)
sym_a_type = symmetry(a_type)
sym_melanoma = symmetry(melanoma)

#box plot of above means
data = [sym_common, sym_a_type, sym_melanoma]
fig = plt.figure(figsize =(7, 7))
ax = fig.add_axes([0, 0, 1, 1])
ax.set_title('Mean Values for Symmetry of Common, A-Type and Melanoma')
ax.set_xticklabels(['Common', 'A-Type', 'Melanoma'])
bp = ax.boxplot(data)
plt.show()

mean_sym_com = np.mean(sym_common)
mean_sym_atype = np.mean(sym_a_type)
mean_sym_melanoma = np.mean(sym_melanoma)

#acuracy for symmetry
sym_accuracy_common = accuracy(sym_common, [mean_sym_com, mean_sym_atype, mean_sym_melanoma])
sym_accuracy_a_type = accuracy(sym_a_type, [mean_sym_com, mean_sym_atype, mean_sym_melanoma])
sym_accuracy_melanoma = accuracy(sym_melanoma, [mean_sym_com, mean_sym_atype, mean_sym_melanoma])

print('Accuracy for Common Nevus: ', (sym_accuracy_common[0]/len(sym_common)) * 100, '%')
print('Accuracy for A-Type Nevus: ', (sym_accuracy_a_type[1]/len(sym_a_type))*100, '%')
print('Accuracy for Melanoma: ', (sym_accuracy_melanoma[2]/len(sym_melanoma))*100, '%')



In [ ]:
#Irregularity
common_irregularity = border_irregularity(common_nevus)
a_type_irregularity = border_irregularity(a_type)
melanoma_irregularity = border_irregularity(melanoma)

#box plot of above means
data = [common_irregularity, a_type_irregularity, melanoma_irregularity]
fig = plt.figure(figsize =(7, 7))
ax = fig.add_axes([0, 0, 1, 1])
ax.set_title('Mean Values for Irregularity of Common, A-Type and Melanoma')
ax.set_xticklabels(['Common', 'A-Type', 'Melanoma'])
bp = ax.boxplot(data)
plt.show()

mean_irregularity_com = np.mean(common_irregularity)
mean_irregularity_atype = np.mean(a_type_irregularity)
mean_irregularity_melanoma = np.mean(melanoma_irregularity)

print(mean_irregularity_com, mean_irregularity_atype, mean_irregularity_melanoma)

#acuracy for irregularity
ir_accuracy_common = accuracy(common_irregularity, [mean_irregularity_com, mean_irregularity_atype, mean_irregularity_melanoma])
ir_accuracy_a_type = accuracy(a_type_irregularity, [mean_irregularity_com, mean_irregularity_atype, mean_irregularity_melanoma])
ir_accuracy_melanoma = accuracy(melanoma_irregularity, [mean_irregularity_com, mean_irregularity_atype, mean_irregularity_melanoma])

print('Accuracy for Common Nevus: ', (ir_accuracy_common[0]/len(common_irregularity)) * 100, '%')
print('Accuracy for A-Type Nevus: ', (ir_accuracy_a_type[1]/len(a_type_irregularity))*100, '%')
print('Accuracy for Melanoma: ', (ir_accuracy_melanoma[2]/len(melanoma_irregularity))*100, '%')


In [ ]:
def color_variance(img_dict_lesion, img_dict_normal):
    color_occur = []
    for (key_lesion, value_lesion), (key_color, value_color) in zip(img_dict_lesion.items(), img_dict_normal.items()):
        image = cv.imread(value_color)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        lesion_image = cv.imread(value_lesion, -1)
        lesion2_image = cv.cvtColor(lesion_image, cv.COLOR_GRAY2RGB)
    
        lesion2_image[lesion_image == 255] = 1,1,1

        final_image = np.multiply(image, lesion2_image)

        mask = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])

        sharpened_image = cv.filter2D(final_image, -1, mask)

        total_colors = set(tuple(pixel) for row in sharpened_image for pixel in row)
        color_variation = len(total_colors)

        color_occur.append(color_variation)

    return color_occur

In [ ]:
#Color Variance
common_color_occurence = color_variance(common_nevus, common_nevus_color)
a_type_color_occurence = color_variance(a_type, a_type_color)
melanoma_color_occurence = color_variance(melanoma, melanoma_color)

#box plot of above means
data = [common_color_occurence, a_type_color_occurence, melanoma_color_occurence]
fig = plt.figure(figsize =(7, 7))
ax = fig.add_axes([0, 0, 1, 1])
ax.set_title('Mean Values for Color Variance of Common, A-Type and Melanoma')
ax.set_xticklabels(['Common', 'A-Type', 'Melanoma'])
bp = ax.boxplot(data)
plt.show()

mean_color_com = np.mean(common_color_occurence)
mean_color_atype = np.mean(a_type_color_occurence)
mean_color_melanoma = np.mean(melanoma_color_occurence)


#acuracy for color variance
color_accuracy_common = accuracy(common_color_occurence, [mean_color_com, mean_color_atype, mean_color_melanoma])
color_accuracy_a_type = accuracy(a_type_color_occurence, [mean_color_com, mean_color_atype, mean_color_melanoma])
color_accuracy_melanoma = accuracy(melanoma_color_occurence, [mean_color_com, mean_color_atype, mean_color_melanoma])

print('Accuracy for Common Nevus: ', (color_accuracy_common[0]/len(common_color_occurence)) * 100, '%')
print('Accuracy for A-Type Nevus: ', (color_accuracy_a_type[1]/len(a_type_color_occurence))*100, '%')
print('Accuracy for Melanoma: ', (color_accuracy_melanoma[2]/len(melanoma_color_occurence))*100, '%')


In [ ]:
mean_accuracy = (dia_accuracy_melanoma+sym_accuracy_melanoma+ir_accuracy_melanoma+color_accuracy_melanoma)/4